In [17]:
import pandas as pd

df = pd.read_excel("BASE.xls", sheet_name=6)

col_dias = "FECHA DE PAGO "
df[col_dias] = pd.to_numeric(df[col_dias], errors="coerce")
fecha_base = pd.to_datetime("2024-04-01")
df["FECHA_REAL"] = fecha_base + pd.to_timedelta(df[col_dias], unit="D")

fecha_real_col = df["FECHA_REAL"].copy()

columnas_a_eliminar = [
    "RET IVA4", "RET ISR_1", "TOTAL_2", "CAMBIOS",
    "IVA3", "RET IVA4", "RET ISR.1","MES","Column1", "SUBTOTAL2", "Unnamed: 36"
]
df.drop(columns=columnas_a_eliminar, inplace=True, errors="ignore")

df_cost_report = pd.read_excel("BASE.xls", sheet_name="Cost Report")
mapeo_cuentas = df_cost_report[["Unnamed: 6", "Unnamed: 24"]].copy()
mapeo_cuentas.columns = ["Subcuenta", "Cuenta_General"]
mapeo_cuentas = mapeo_cuentas.dropna()
mapeo_cuentas["Subcuenta"] = mapeo_cuentas["Subcuenta"].astype(str).str.strip()
mapeo_cuentas["Cuenta_General"] = mapeo_cuentas["Cuenta_General"].astype(str).str.strip()

df["Valida"] = df["Valida"].astype(str).str.strip()
df_enriquecido = df.merge(mapeo_cuentas, left_on="Valida", right_on="Subcuenta", how="left")

df_filtrado = df_enriquecido[["Cuenta_General", "Subcuenta", "TOTAL"]].dropna()
df_filtrado["TOTAL"] = pd.to_numeric(df_filtrado["TOTAL"], errors='coerce')
df_filtrado["Cuenta_General"] = df_filtrado["Cuenta_General"].astype(str).str.strip()

gasto_total_por_cuenta = df_filtrado.groupby("Cuenta_General")["TOTAL"].sum().reset_index()
top_35_cuentas = gasto_total_por_cuenta.nlargest(35, "TOTAL")

cuentas_top = top_35_cuentas["Cuenta_General"].astype(str).str.strip().tolist()
df = df_enriquecido[df_enriquecido["Cuenta_General"].isin(cuentas_top)].copy()



df["FECHA_REAL"] = fecha_real_col
df.reset_index(drop=True, inplace=True)

print("Top 35 cuentas generales:")
print(top_35_cuentas)


df

/opt/anaconda3/envs/IDM/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Top 35 cuentas generales:
    Cuenta_General         TOTAL
183         7003.0  1.057418e+07
13          1401.0  9.315454e+06
115         3193.0  8.773958e+06
0           1101.0  8.750142e+06
139         3683.0  6.974556e+06
5           1201.0  6.239016e+06
146         3902.0  5.206968e+06
128         3410.0  5.182840e+06
129         3412.0  4.702875e+06
176         6004.0  3.593923e+06
70          2494.0  3.136409e+06
51          2103.0  2.368564e+06
10          1301.0  2.162454e+06
158         5022.0  2.016080e+06
24          1499.0  1.834947e+06
88          2794.0  1.690597e+06
151         4180.0  1.598079e+06
152         5001.0  1.576253e+06
190         7201.0  1.545947e+06
14          1403.0  1.545614e+06
138         3632.0  1.466134e+06
161         5105.0  1.367640e+06
2           1107.0  1.096791e+06
6           1202.0  1.092537e+06
131         3442.0  1.064416e+06
105         3101.0  1.032012e+06
142         3793.0  1.020800e+06
7           1204.0  1.004575e+06
184         7007.

,CONTROL,# CHEQUE,PARTIDA PRESUPUESTAL,UIDD/FOLIO FISCAL,DIRECCION,REGIMEN FISCAL,NOMBRE REGIMEN FISCAL,RFC,STATUS,FECHA DE RECIBIDO,...,CUENTA CLAVE,BANCO,ESTATUS,FECHA DE PAGO,NOTAS,Valida,TOTAL.1,FECHA_REAL,Subcuenta,Cuenta_General
0,N001,0001,7007-009,0.0,86127,Régimen Simplificado de Confianza,Régimen Simplificado de Confianza,1.0,NaN,33.0,...,0.0,BANAMEX,Pagado,37.0,NaN,7007-009,NaN,2024-05-08,7007-009,7007.0
1,N003,0004,1107-009,1.0,4318,Régimen Simplificado de Confianza,Régimen Simplificado de Confianza,2.0,NaN,40.0,...,1.0,BANCOMER,Pagado,49.0,NaN,1107-009,NaN,2024-05-20,1107-009,1107.0
2,N005,0006,1101-007,3.0,4318,Régimen Simplificado de Confianza,Régimen Simplificado de Confianza,2.0,NaN,44.0,...,1.0,BANCOMER,Pagado,55.0,NaN,1101-007,NaN,2024-05-26,1101-007,1101.0
3,N006,0007,1107-005,4.0,6140,Personas Físicas con Actividades Empresariales...,Personas Físicas con Actividades Empresariales...,4.0,NaN,44.0,...,3.0,BANCOMER,Pagado,55.0,NaN,1107-005,NaN,2024-05-26,1107-005,1107.0
4,N007,0008,1101-006,5.0,6140,Personas Físicas con Actividades Empresariales...,Personas Físicas con Actividades Empresariales...,4.0,NaN,48.0,...,3.0,BANCOMER,Pagado,58.0,NaN,1101-006,NaN,2024-05-29,1101-006,1101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,NaN,NaN,3410-012,4614.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3410-012,NaN,NaT,3410-012,3410.0
3191,NaN,NaN,3410-012,4614.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3410-012,NaN,NaT,3410-012,3410.0
3192,NaN,NaN,4180-003,4614.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4180-003,NaN,NaT,4180-003,4180.0
3193,NaN,NaN,3410-012,4614.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3410-012,NaN,NaT,3410-012,3410.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CONTROL                2294 non-null   object        
 1   # CHEQUE               2679 non-null   object        
 2   PARTIDA PRESUPUESTAL   3195 non-null   object        
 3   UIDD/FOLIO FISCAL      3173 non-null   float64       
 4   DIRECCION              2888 non-null   object        
 5   REGIMEN FISCAL         2900 non-null   object        
 6   NOMBRE REGIMEN FISCAL  2878 non-null   object        
 7   RFC                    2928 non-null   float64       
 8   STATUS                 2868 non-null   object        
 9   FECHA DE RECIBIDO      465 non-null    float64       
 10  SEMANA QUE SE PAGA     3044 non-null   object        
 11  TIPO                   3192 non-null   object        
 12  NOMBRE DEL PROVEEDOR   3192 non-null   float64       
 13  COR

In [58]:
DF=pd.read_excel("BASE.xls", sheet_name="Data")
DF[DF["PARTIDA PRESUPUESTAL"] == "1104-003"]  # Filtra en el DataFrame original

/opt/anaconda3/envs/IDM/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,CONTROL,# CHEQUE,PARTIDA PRESUPUESTAL,UIDD/FOLIO FISCAL,DIRECCION,REGIMEN FISCAL,NOMBRE REGIMEN FISCAL,RFC,STATUS,FECHA DE RECIBIDO,...,Valida,MES,Column1,SUBTOTAL2,IVA3,RET IVA4,RET ISR.1,TOTAL.1,CAMBIOS,Unnamed: 36
36,N021,0031,1104-003,22.0,1729,Personas Físicas con Actividades Empresariales...,Personas Físicas con Actividades Empresariales...,9.0,NaN,97.0,...,1104-003,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,-45071.0
81,N051,64,1104-003,56.0,NaN,Personas Físicas con Actividades Empresariales...,Personas Físicas con Actividades Empresariales...,9.0,NaN,NaN,...,1104-003,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_Data =DF[["SUBTOTAL", "PARTIDA PRESUPUESTAL"]].copy()  # Copia explícita
df_Data[df_Data["PARTIDA PRESUPUESTAL"] == "1104-003"] # Verifica si hay datos

,SUBTOTAL,PARTIDA PRESUPUESTAL
36,100000.0,1104-003
81,50000.0,1104-003


In [60]:
df_Data.to_csv("Data.csv", index=False)